In [0]:
# Select TensorFlow 2.0
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
# Install Weights and Biases
!pip install wandb

In [0]:
# Imports
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten
from tensorflow.keras import utils
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import TensorBoard

import wandb
from wandb.keras import WandbCallback

In [0]:
# Login to your W&B account
!wandb login

In [0]:
# Initialize your W&B project allowing it to sync with TensorBoard
wandb.init(project="tensorboard-integration", sync_tensorboard=True)
config = wandb.config

In [0]:
# Specify the configuration variables
config.dropout = 0.2
config.hidden_layer_size = 128
config.layer_1_size  = 16
config.layer_2_size = 32
config.learn_rate = 0.01
config.decay = 1e-6
config.momentum = 0.9
config.epochs = 25

In [0]:
# Load and preprocess the data
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
labels=["T-shirt/top","Trouser","Pullover","Dress","Coat",
        "Sandal","Shirt","Sneaker","Bag","Ankle boot"]

img_width=28
img_height=28

X_train = X_train.astype('float32')
X_train /= 255.
X_test = X_test.astype('float32')
X_test /= 255.

# Reshape input data
X_train = X_train.reshape(X_train.shape[0], img_width, img_height, 1)
X_test = X_test.reshape(X_test.shape[0], img_width, img_height, 1)

# One hot encode outputs
y_train = utils.to_categorical(y_train)
y_test = utils.to_categorical(y_test)
num_classes = y_test.shape[1]

4423680/4422102 [==============================] - 0s 0us/step


In [0]:
# Train the model
sgd = SGD(lr=config.learn_rate, decay=config.decay, momentum=config.momentum,
                            nesterov=True)

model = Sequential()
model.add(Conv2D(config.layer_1_size, (5, 5), activation='relu',
                            input_shape=(img_width, img_height,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(config.layer_2_size, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(config.dropout))
model.add(Flatten())
model.add(Dense(config.hidden_layer_size, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# The WandbCallback logs metrics and some examples of the test data
model.fit(X_train, y_train,  validation_data=(X_test, y_test), epochs=config.epochs,
    callbacks=[WandbCallback(data_type="image", labels=labels), 
               TensorBoard(log_dir=wandb.run.dir)])

Train on 60000 samples, validate on 10000 samples
Epoch 1/25
60000/60000 [==============================] - 40s 664us/sample - loss: 0.5759 - accuracy: 0.7878 - val_loss: 0.3768 - val_accuracy: 0.8607
Epoch 2/25
60000/60000 [==============================] - 37s 615us/sample - loss: 0.3774 - accuracy: 0.8621 - val_loss: 0.3495 - val_accuracy: 0.8723
Epoch 3/25
60000/60000 [==============================] - 37s 612us/sample - loss: 0.3355 - accuracy: 0.8769 - val_loss: 0.3151 - val_accuracy: 0.8856
Epoch 4/25
60000/60000 [==============================] - 35s 585us/sample - loss: 0.3108 - accuracy: 0.8855 - val_loss: 0.2991 - val_accuracy: 0.8926
Epoch 5/25
60000/60000 [==============================] - 35s 579us/sample - loss: 0.2919 - accuracy: 0.8911 - val_loss: 0.3040 - val_accuracy: 0.8926
Epoch 6/25
60000/60000 [==============================] - 35s 579us/sample - loss: 0.2787 - accuracy: 0.8956 - val_loss: 0.2912 - val_accuracy: 0.8924
Epoch 7/25
60000/60000 [====================

Run page: https://app.wandb.ai/sayakpaul/tensorboard-integration/runs/e8kv5zab